# TITLE

"SOBRECARGA DE TRABAJO EN LOS CENTROS DE SERVICIOS SOCIALES EN EL AYUNTAMIENTO DE MADRID" 

## SUBJECT:

"VALORAR LA SOBREEXPLOTACION DE LOS TRABAJADORES SOCIALES EN EPOCA POST-COVID"

Los datos a valorar son los del año 2021, que es un año completo.

Sería interesante ver los datos en los años 2019 y 2020 para valorar como ha ido evolucionando en estos años

### HYPOTHESIS:
hipotesis_1 = "Hay mucha demanda de ayudas y servicios sociales en determinados distritos (valorar si es en todos) y poco personal para absorber esta demanda"

hipotesis_2 = "Valorar si hay suficientes recursos no humanos para atender las demandas de los usuarios"

### 1.- Data Collection:

La fuente principal de todos los datos obtenidos ha sido a través de las páginas del ayuntamiento de Madrid (MUNIMADRID y PORTAL DE DATOS ABIERTOS DEL AYUNTAMIENTO DE MADRID)


In [ ]:
# guarda en variables los datasets y su fuente
# df_1 = pd.DataFrame()
# URL_indice_vulnerabilidad_rentas="https://datos.madrid.es/sites/v/index.jsp?vgnextoid=bbd71fa5c48a0810VgnVCM2000001f4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD""
##  DE ESTE VINCULO (Esfera de Economía y Empleo por Distritos y Barrios) SE EXTRAEN DATOS DE RENTA MEDIA POR HOGAR

#df_2 = pd.DataFrame()
# Servicios Sociales Ayto
## ESTE ES UN FICHERO EXTRAIDO DE DATOS.MADRID, DEL CUAL NO TENGO VINCULO. EN EL SE DETALLA EL NUMERO DE TRABAJADORES POR CENTRO Y DISTRITO

#df_3 = pd.DataFrame()
# URL_personas_atendidas="https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=8c936c7fe412b610VgnVCM2000001f4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default"
# SE DETALLAN LAS PERSONAS ATENDIDAS

#df_4 = pd.DataFrame()
# URL_citas_proporcionadas="https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=73ee23d869578610VgnVCM2000001f4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default"
# SE DETALLAN LAS CITAS QUE SE HAN DADO A USUARIOS, RELACIONADA CON EL DF_3

#df_5 = pd.DataFrame()
# URL_demandas_de_servicio="https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=40ab4e8b0327d610VgnVCM1000001d4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default"
# SOLICITUDES DE PROCESOS SOLICITADOS POR LOS USUARIOS

#df_6 = pd.DataFrame()
# URL_urgencias_atendidas="https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=0b006dace9578610VgnVCM1000001d4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default"
# URGENCIAS ATENDIDAS EN LOS CENTROS. ESTAS URGENCIAS SON DE LA SIGUIENTES TIPOLOGIAS 
# Abandono
# Alojamiento/Vivienda
# Desprotección social problema salud
# Desprotección socio-familiar
# Malos tratos

#df_7 = pd.DataFrame()
# URL: http://www-2.munimadrid.es/CSE6/control/mostrarDatos
## ESTE ES UN FICHERO EXTRAIDO DE MUNIMADRID, EL CUAL ES UN FICHERO DE EXCEL DONDE EXTRAIGO LA POBLACION DE CADA DISTRITO


# OTROS POSIBLES DF QUE PUDIERA NECESITAR:
#URL_datos_abiertos_madrid="https://datos.madrid.es/portal/site/egob/menuitem.9e1e2f6404558187cf35cf3584f1a5a0/?vgnextoid=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default"
#URL_centros_sociales="https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=7c123e08dafc8410VgnVCM1000000b205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default"
#URL_ayudas_sociales="https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=00937b777f8ea610VgnVCM2000001f4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default"
#URL_panel_indicador="https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=71359583a773a510VgnVCM2000001f4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default"
#URL_ranking_vulnerabilidad="https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=d029ed1e80d38610VgnVCM2000001f4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default"


### 2.- Data Extraction from different Datasets and storage in data\Outcomes

### 3.- Featuring Engineering

### 4.- Exploratory Data Analysis

### 5.- Graphics

## CONCLUSIÓN: